<a href="https://colab.research.google.com/github/emichester/06MIAR_Aprendizaje_No_Supervisado/blob/main/MyTests/Tyre_Quality_DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive mount

In [1]:
# montamos la unidad drive donde tenemos los datos en la carpeta drive/My Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/Colab Notebooks/01_ML/00_Tyre_Quality"

/content/drive/MyDrive/Colab Notebooks/01_ML/00_Tyre_Quality


In [3]:
_base_path = "/content/drive/MyDrive/Colab Notebooks/01_ML/00_Tyre_Quality"

# Kaggle ops

In [4]:
!pip install -q kaggle

In [5]:
# from google.colab import files
# files.upload() # upload the kaggle token kaggle.json

In [6]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets list

ref                                                        title                                          size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------  --------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
nelgiriyewithana/top-spotify-songs-2023                    Most Streamed Spotify Songs 2023               47KB  2023-08-26 11:04:57          14167        452  1.0              
joebeachcapital/students-performance                       Students Performance                            2KB  2023-08-31 00:50:11           4911        130  1.0              
iamsouravbanerjee/airline-dataset                          Airline Dataset                                 8MB  2023-09-13 06:47:17           5375        122  1.0              
carlmcbrideellis/zzzs-lightweight-training-dataset-target  Zzzs: Lightweight training dataset + target   185MB  202

In [8]:
# !kaggle datasets download -d nomihsa965/large-scale-image-dataset-of-wood-surface-defects

In [9]:
# !kaggle datasets download -d warcoder/tyre-quality-classification

In [10]:
# %mkdir -p "/content/drive/MyDrive/Colab Notebooks/01_ML/00_Tyre_Quality/kaggle/input"

In [11]:
# %mkdir "/content/drive/MyDrive/Colab Notebooks/01_ML/00_Tyre_Quality/kaggle/input/tyre-quality-classification"

In [12]:
# !unzip "/content/drive/MyDrive/Colab Notebooks/01_ML/00_Tyre_Quality/tyre-quality-classification.zip" -d "/content/drive/MyDrive/Colab Notebooks/01_ML/00_Tyre_Quality/kaggle/input/tyre-quality-classification"

In [13]:
%ls

checkpoints/         generated_plots/         kaggle/
checkpoints_before/  generated_plots_before/  kaggle.json
checkpoints_old_v2/  generated_plots_old_v2/  tyre-quality-classification.zip


In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import glob
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import  ImageDataGenerator
from keras.layers import Dense,Flatten,BatchNormalization,MaxPooling2D,Conv2D,Input
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras import Model
from sklearn.metrics import classification_report , confusion_matrix

# import os
# for dirname, _, filenames in os.walk('./kaggle/input'):
#     print (dirname)

./kaggle/input
./kaggle/input/tyre-quality-classification
./kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres
./kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/defective
./kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/good
./kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/defective_resized_196
./kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/defective_resized


In [15]:
# good = glob.glob(r'./kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/good/*.jpg')
# defective = glob.glob(r'./kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/defective/*.jpg')

# random.shuffle(good)
# random.shuffle(defective)

# print ('number of good tyres : {} images'.format(len (good)))
# print ('number of defective tyres : {} images'.format(len (defective)))

In [16]:
# train_good = good [:int (len (good) *0.8)]
# test_good = good[int (len (good) *0.8):]

# train_defective = defective [:int (len (defective) *0.8)]
# test_defective = defective[int (len (defective) *0.8):]

# print ('number of train good tyres : {} images'.format(len (train_good)))
# print ('number of test good tyres : {} images'.format(len (test_good)))
# print ('number of train defective tyres : {} images'.format(len (train_defective)))
# print ('number of test defective tyres : {} images'.format(len (test_defective)))

In [17]:
_path_folder = "./kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/defective"
# _all_images = glob.glob(_path_folder+"/*.jpg")
# img = cv2.imread(_all_images[60])
# print(img.shape)
# # plt.imshow(img);
# plt.figure()
# # plt.imshow(cv2.resize(img, (192,192), interpolation=cv2.INTER_AREA));
# plt.imshow(cv2.resize(img, (256,256), interpolation=cv2.INTER_AREA));

In [18]:
# %mkdir -p "./kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/defective_resized"

In [19]:
from IPython.display import HTML, display
import time

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

In [20]:
_path_folder_resized = "./kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/defective_resized"

# _max = len(glob.glob(_path_folder+"/*.jpg"))
# out = display(progress(0, _max), display_id=True)
# for ii,img_name in enumerate(glob.glob(_path_folder+"/*.jpg")):
#   img = cv2.imread(img_name)
#   img = cv2.resize(img, (256,256))
#   cv2.imwrite(os.path.join(_path_folder_resized,os.path.basename(img_name)), img)
#   out.update(progress(ii, _max))

## Código estudiado

## Mejoras
- [The architecture of DCGAN](https://www.researchgate.net/figure/The-architecture-of-DCGAN-a-generator-network-b-architecture-of-DCGAN-c-and_fig6_341605614)
- [Deep Convolutional GAN](https://paperswithcode.com/method/dcgan)
- [DCGAN-Keras](https://github.com/mitchelljy/DCGAN-Keras)

In [21]:
IMAGES_PATH = _path_folder_resized+"/*.jpg"
%mkdir -p "./checkpoints/"
%mkdir -p "./generated_plots/"

In [22]:
# importamos las librerías necesarias
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout
# configuramos Colab para que nos muestre las imágenes más grandes
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,10)

In [23]:
# # definimos el generador
# def define_generator(latent_dim):
#     model = Sequential(name="GENERATOR")
#     n_nodes = 256 * 24 * 24
#     model.add(Dense(n_nodes, input_dim=latent_dim))
#     model.add(LeakyReLU(alpha=0.2))
#     model.add(Reshape((24, 24, 256)))
#     # upsample a 8x8
#     model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
#     model.add(LeakyReLU(alpha=0.2))
#     # upsample a 16x16
#     model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
#     model.add(LeakyReLU(alpha=0.2))
#     # upsample a 32x32
#     model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
#     model.add(LeakyReLU(alpha=0.2))
#     # salida (nuestra imagen fake)
#     model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
#     model.summary()
#     return model
# definimos el generador
def define_generator(latent_dim):
    model = Sequential(name="GENERATOR")
    n_nodes = 256 * 2 * 2
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((2, 2, 256)))
    # upsample
    model.add(Conv2DTranspose(260, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample
    model.add(Conv2DTranspose(128, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample
    model.add(Conv2DTranspose(64, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample
    model.add(Conv2DTranspose(32, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample
    model.add(Conv2DTranspose(16, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample
    model.add(Conv2DTranspose(8, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample
    model.add(Conv2DTranspose(4, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # salida (nuestra imagen fake)
    model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
    model.summary()
    return model

In [53]:
# # definimos el discriminador
# def define_discriminator(in_shape=(192,192,3)):
#     model = Sequential(name="DISCRIMINATOR")
#     # normal
#     model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
#     model.add(LeakyReLU(alpha=0.2))
#     # downsample (por el atributo 'strides')
#     model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
#     model.add(LeakyReLU(alpha=0.2))
#     # downsample
#     model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
#     model.add(LeakyReLU(alpha=0.2))
#     # downsample
#     model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
#     model.add(LeakyReLU(alpha=0.2))
#     # clasificador
#     model.add(Flatten())
#     model.add(Dropout(0.4))
#     model.add(Dense(1, activation='sigmoid'))
#     # compilamos modelo
#     opt = Adam(learning_rate=0.0002, beta_1=0.5)
#     model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
#     model.summary()
#     return model
# definimos el discriminador
def define_discriminator(in_shape=(256,256,3)):
    model = Sequential(name="DISCRIMINATOR")
    # additional
    model.add(Conv2D(4, (3,3), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(8, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # additional
    model.add(Conv2D(16, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # normal
    model.add(Conv2D(32, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample (por el atributo 'strides')
    model.add(Conv2D(64, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(260, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # clasificador
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    # compilamos modelo
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    # opt = Adam(learning_rate=0.01, beta_1=0.5)
    # model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.summary()
    return model

In [54]:
# definimos el modelo GAN combinando generador y discriminador, para entrenar el generador
def define_gan(g_model, d_model):
    # Así que congelamos el discriminador:
    d_model.trainable = False
    # ahora conectamos el G(z) al D(x)
    model = Sequential()
    # añadimos el generador primero: él es el encargado de generar una muestra
    # a partir del espacio latente
    model.add(g_model)
    # y el discriminador después: le introducimos la muestra generada por el
    # G(z) para que nos diga si cree que es real o fake
    model.add(d_model)
    # y ahora sí, compilamos el modelo
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))
    # model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.01, beta_1=0.5))
    return model

In [55]:
def load_imgs(image_path=IMAGES_PATH):
    X_train = []
    for i in glob.glob(image_path):
        # img = Image.open(i)
        # img = np.asarray(img)
        img = cv2.imread(i)
        X_train.append(img)
    return np.asarray(X_train)

# definimos las funciones para cargar el MNIST
def load_real_samples():
    # cargamos el cifar10
    trainX = load_imgs()
    # convertimos a float32
    X = trainX.astype('float32')
    # escalamos entre -1 y 1 porque hemos puesto la función de activación
    # del generador tangente hiperbólica
    X = (X - 127.5) / 127.5
    return X

# nos creamos una función que nos devuelva n_samples del dataset con sus
# etiquetas (1)
def generate_real_samples(dataset, n_samples):
    # seleccionamos n_samples muestras aleatoriamente
    ix = np.random.randint(0, dataset.shape[0], n_samples)
    # las cogemos
    X = dataset[ix]
    # generamos las etiquetas reales (1)
    y = np.ones((n_samples, 1))
    return X, y

In [56]:
# generamos los vectores latentes que introduciremos al generador
def generate_latent_points(latent_dim, batch_size):
    # generamos un vector de batch_size * latent_dim números aleatorios
    # latent_dim es la dimensión del vector latente
    # batch_size es el número de elementos por batch
    x_input = np.random.randn(latent_dim * batch_size)
    # redimensionamos el vector para que tenga un tamaño (batch_size, latent_dim)
    x_input = x_input.reshape(batch_size, latent_dim)
    return x_input

# creamos datos fake con el generador (dinero falsificado)
def generate_fake_samples(g_model, latent_dim, n_samples):
    # usamos la función anterior para generar los vectores latentes que
    # necesitamos para generar muestras fake
    x_input = generate_latent_points(latent_dim, n_samples)
    # le introducimos los vectores latentes al generador para obtener
    # muestras similares a las reales
    X = g_model.predict(x_input)
    # le asignamos la etiqueta 0 (porque utilizaremos esta función para
    # entrenar el D)
    y = np.zeros((n_samples, 1))
    return X, y

In [57]:
# función para guardar las imágenes generadas
def save_plot(examples, epoch, n=7):
	# escalamos de [-1,1] (la salida de nuestra gan, debido a la función de activación tanh) a [0,1]
	examples = (examples + 1) / 2.0
	for i in range(n * n):
		plt.subplot(n, n, 1 + i)
		plt.axis('off')
		plt.imshow(examples[i])
	# guardamos las imágenes
	filename = os.path.join(_base_path,'generated_plots','generated_plot_e%03d.png' % (epoch+1))
	plt.savefig(filename)
	plt.close()

In [58]:
# función para entrenar la GAN: el discriminador y el generador
def train(g_model, d_model, gan_model, dataset, latent_dim, start_epoch=0, n_epochs=100, n_batch=128):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # bucle para las epochs
    for epoch in range(start_epoch, n_epochs):
        # bucle para los batch
        for batch in range(bat_per_epo):

            # en esta ocasión vamos a separar las pérdidas del discriminador
            # cuando le metemos imágenes reales y cuando le metemos imágenes
            # fake para ver cómo lo hace con cada tipo
            # recordad que lo ideal es que llegue a un 50% de acc en cada uno

            # preparamos los datos reales
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # actualizamos el discriminador
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)

            # generamos datos falsos
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # actualizamos el discriminador
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)

            # preparamos los puntos en el espacio latente: serán la entrada al
            # modelo GAN con el que entrenaremos el generador
            X_gan = generate_latent_points(latent_dim, n_batch)

            # creamos etiquetas invertidas para el generador: utilizamos el D(x)
            # para que piense que las muestras que le introducimos son reales, y
            # en caso de que diga que no son reales, aprovechamos la información
            # de sus gradientes para actualizar el G(z) para que la próxima vez
            # los datos generados por G(z) sean más plausibles (parecidos a los
            # reales)
            y_gan = np.ones((n_batch, 1))

            # como acabamos de ver, entrenamos el generador de forma que actualice
            # sus pesos usando los gradientes del discriminador
            # tened en cuenta que en este modelo (gan_model) el discriminador está
            # congelado, por lo que no se actualizan sus pesos: no queremos "untar"
            # a nuestro policía, lo que queremos es fabricar dinero más realista.
            g_loss = gan_model.train_on_batch(X_gan, y_gan)

            # mostramos el progreso
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
                  (epoch+1, batch+1, bat_per_epo, d_loss1, d_loss2, g_loss))
        # evaluate the model performance, sometimes
        if (epoch+1) % 10 == 0 or epoch == 0:
            # preparamos ejemplos reales
            X_real, y_real = generate_real_samples(dataset, n_batch)
            # evaluamos el discriminador con datos reales
            _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
            # preparamos ejemplos fake
            x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_batch)
            # evaluamos el discriminador con datos fake
            _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
            # mostramos cómo de bueno es nuestro policía
            print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
            # guardamos las imágenes generadas
            save_plot(x_fake, epoch)
            # guardamos el generador para tenerlo disponible más tarde
            filename = os.path.join(_base_path,'checkpoints','generator_model_%03d.h5' % (epoch + 1))
            g_model.save(filename)

## Training

In [59]:
# load image data
dataset = load_real_samples()

KeyboardInterrupt: ignored

In [62]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# si ya existe un checkpoint continúa
# create the generator
g_model = define_generator(latent_dim)
if len(sorted(os.listdir("./checkpoints/"))) > 0:
    _last_checkpoint = sorted(os.listdir("./checkpoints/"))[-1]
    _start_epoch = int(_last_checkpoint.split("_")[2].split(".")[0])+1
    g_model.load_weights(os.path.join("./checkpoints/",_last_checkpoint))
else:
    _start_epoch = 0
# create the gan
gan_model = define_gan(g_model, d_model)
# train model
train(g_model, d_model, gan_model, dataset, latent_dim, start_epoch=_start_epoch, n_epochs=1500, n_batch=64)

Model: "DISCRIMINATOR"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_56 (Conv2D)          (None, 256, 256, 4)       112       
                                                                 
 leaky_re_lu_105 (LeakyReLU  (None, 256, 256, 4)       0         
 )                                                               
                                                                 
 conv2d_57 (Conv2D)          (None, 128, 128, 8)       296       
                                                                 
 leaky_re_lu_106 (LeakyReLU  (None, 128, 128, 8)       0         
 )                                                               
                                                                 
 conv2d_58 (Conv2D)          (None, 64, 64, 16)        1168      
                                                                 
 leaky_re_lu_107 (LeakyReLU  (None, 64, 64, 16)      

1/1 [==============================] - 0s 19ms/step
>461, 1/16, d1=0.620, d2=0.788 g=0.829
1/1 [==============================] - 0s 22ms/step
>461, 2/16, d1=0.610, d2=0.717 g=0.870
1/1 [==============================] - 0s 17ms/step
>461, 3/16, d1=0.642, d2=0.722 g=0.854
1/1 [==============================] - 0s 17ms/step
>461, 4/16, d1=0.635, d2=0.660 g=0.847
1/1 [==============================] - 0s 20ms/step
>461, 5/16, d1=0.571, d2=0.689 g=0.792
1/1 [==============================] - 0s 18ms/step
>461, 6/16, d1=0.595, d2=0.759 g=0.797
1/1 [==============================] - 0s 18ms/step
>461, 7/16, d1=0.594, d2=0.742 g=0.815
1/1 [==============================] - 0s 18ms/step
>461, 8/16, d1=0.624, d2=0.743 g=0.892
1/1 [==============================] - 0s 17ms/step
>461, 9/16, d1=0.609, d2=0.650 g=0.952
1/1 [==============================] - 0s 17ms/step
>461, 10/16, d1=0.671, d2=0.611 g=0.924
1/1 [==============================] - 0s 20ms/step
>461, 11/16, d1=0.614, d2=0.682 g=0.9

1/1 [==============================] - 0s 18ms/step
>471, 1/16, d1=0.573, d2=0.609 g=1.055
1/1 [==============================] - 0s 18ms/step
>471, 2/16, d1=0.550, d2=0.499 g=1.089
1/1 [==============================] - 0s 24ms/step
>471, 3/16, d1=0.615, d2=0.493 g=1.055
1/1 [==============================] - 0s 17ms/step
>471, 4/16, d1=0.549, d2=0.496 g=1.080
1/1 [==============================] - 0s 17ms/step
>471, 5/16, d1=0.524, d2=0.521 g=1.015
1/1 [==============================] - 0s 19ms/step
>471, 6/16, d1=0.481, d2=0.567 g=0.933
1/1 [==============================] - 0s 19ms/step
>471, 7/16, d1=0.464, d2=0.628 g=0.982
1/1 [==============================] - 0s 18ms/step
>471, 8/16, d1=0.483, d2=0.564 g=0.994
1/1 [==============================] - 0s 17ms/step
>471, 9/16, d1=0.488, d2=0.564 g=0.959
1/1 [==============================] - 0s 23ms/step
>471, 10/16, d1=0.495, d2=0.611 g=0.899
1/1 [==============================] - 0s 24ms/step
>471, 11/16, d1=0.517, d2=0.623 g=0.9

1/1 [==============================] - 0s 27ms/step
>481, 1/16, d1=0.729, d2=0.601 g=0.960
1/1 [==============================] - 0s 28ms/step
>481, 2/16, d1=0.676, d2=0.634 g=0.934
1/1 [==============================] - 0s 28ms/step
>481, 3/16, d1=0.669, d2=0.620 g=0.912
1/1 [==============================] - 0s 24ms/step
>481, 4/16, d1=0.649, d2=0.726 g=0.994
1/1 [==============================] - 0s 25ms/step
>481, 5/16, d1=0.692, d2=0.596 g=1.043
1/1 [==============================] - 0s 27ms/step
>481, 6/16, d1=0.804, d2=0.495 g=1.116
1/1 [==============================] - 0s 28ms/step
>481, 7/16, d1=0.741, d2=0.545 g=1.043
1/1 [==============================] - 0s 17ms/step
>481, 8/16, d1=0.770, d2=0.570 g=0.971
1/1 [==============================] - 0s 19ms/step
>481, 9/16, d1=0.721, d2=0.597 g=0.908
1/1 [==============================] - 0s 18ms/step
>481, 10/16, d1=0.617, d2=0.633 g=0.945
1/1 [==============================] - 0s 19ms/step
>481, 11/16, d1=0.655, d2=0.572 g=0.9

1/1 [==============================] - 0s 24ms/step
>491, 1/16, d1=0.598, d2=0.623 g=0.962
1/1 [==============================] - 0s 26ms/step
>491, 2/16, d1=0.706, d2=0.733 g=0.980
1/1 [==============================] - 0s 24ms/step
>491, 3/16, d1=0.760, d2=0.655 g=0.935
1/1 [==============================] - 0s 28ms/step
>491, 4/16, d1=0.788, d2=0.690 g=0.871
1/1 [==============================] - 0s 26ms/step
>491, 5/16, d1=0.618, d2=0.733 g=0.918
1/1 [==============================] - 0s 28ms/step
>491, 6/16, d1=0.629, d2=0.699 g=0.868
1/1 [==============================] - 0s 24ms/step
>491, 7/16, d1=0.694, d2=0.733 g=0.990
1/1 [==============================] - 0s 16ms/step
>491, 8/16, d1=0.636, d2=0.682 g=0.948
1/1 [==============================] - 0s 17ms/step
>491, 9/16, d1=0.702, d2=0.602 g=0.952
1/1 [==============================] - 0s 19ms/step
>491, 10/16, d1=0.810, d2=0.652 g=0.881
1/1 [==============================] - 0s 18ms/step
>491, 11/16, d1=0.641, d2=0.694 g=0.9

...

1/1 [==============================] - 0s 16ms/step
>591, 1/16, d1=0.571, d2=0.667 g=0.882
1/1 [==============================] - 0s 16ms/step
>591, 2/16, d1=0.639, d2=0.634 g=0.957
1/1 [==============================] - 0s 20ms/step
>591, 3/16, d1=0.590, d2=0.583 g=0.922
1/1 [==============================] - 0s 17ms/step
>591, 4/16, d1=0.632, d2=0.594 g=0.950
1/1 [==============================] - 0s 17ms/step
>591, 5/16, d1=0.607, d2=0.574 g=0.963
1/1 [==============================] - 0s 19ms/step
>591, 6/16, d1=0.619, d2=0.601 g=0.942
1/1 [==============================] - 0s 17ms/step
>591, 7/16, d1=0.644, d2=0.585 g=0.962
1/1 [==============================] - 0s 17ms/step
>591, 8/16, d1=0.561, d2=0.622 g=1.011
1/1 [==============================] - 0s 19ms/step
>591, 9/16, d1=0.654, d2=0.562 g=1.017
1/1 [==============================] - 0s 17ms/step
>591, 10/16, d1=0.694, d2=0.551 g=1.008
1/1 [==============================] - 0s 18ms/step
>591, 11/16, d1=0.623, d2=0.571 g=0.9

1/1 [==============================] - 0s 35ms/step
>601, 1/16, d1=0.695, d2=0.501 g=0.964
1/1 [==============================] - 0s 27ms/step
>601, 2/16, d1=0.517, d2=0.683 g=0.949
1/1 [==============================] - 0s 26ms/step
>601, 3/16, d1=0.570, d2=0.629 g=1.018
1/1 [==============================] - 0s 30ms/step
>601, 4/16, d1=0.710, d2=0.659 g=0.997
1/1 [==============================] - 0s 26ms/step
>601, 5/16, d1=0.670, d2=0.619 g=0.916
1/1 [==============================] - 0s 29ms/step
>601, 6/16, d1=0.651, d2=0.636 g=0.897
1/1 [==============================] - 0s 26ms/step
>601, 7/16, d1=0.595, d2=0.613 g=0.913
1/1 [==============================] - 0s 17ms/step
>601, 8/16, d1=0.534, d2=0.722 g=1.006
1/1 [==============================] - 0s 17ms/step
>601, 9/16, d1=0.618, d2=0.755 g=1.035
1/1 [==============================] - 0s 24ms/step
>601, 10/16, d1=0.789, d2=0.551 g=1.064
1/1 [==============================] - 0s 17ms/step
>601, 11/16, d1=0.773, d2=0.580 g=1.0

1/1 [==============================] - 0s 16ms/step
>611, 1/16, d1=0.652, d2=0.641 g=0.846
1/1 [==============================] - 0s 16ms/step
>611, 2/16, d1=0.627, d2=0.691 g=0.817
1/1 [==============================] - 0s 17ms/step
>611, 3/16, d1=0.676, d2=0.657 g=0.805
1/1 [==============================] - 0s 17ms/step
>611, 4/16, d1=0.626, d2=0.627 g=0.823
1/1 [==============================] - 0s 17ms/step
>611, 5/16, d1=0.617, d2=0.666 g=0.784
1/1 [==============================] - 0s 17ms/step
>611, 6/16, d1=0.624, d2=0.655 g=0.852
1/1 [==============================] - 0s 17ms/step
>611, 7/16, d1=0.627, d2=0.658 g=0.780
1/1 [==============================] - 0s 22ms/step
>611, 8/16, d1=0.623, d2=0.623 g=0.797
1/1 [==============================] - 0s 17ms/step
>611, 9/16, d1=0.644, d2=0.671 g=0.802
1/1 [==============================] - 0s 18ms/step
>611, 10/16, d1=0.715, d2=0.711 g=0.856
1/1 [==============================] - 0s 17ms/step
>611, 11/16, d1=0.700, d2=0.579 g=0.8

1/1 [==============================] - 0s 30ms/step
>621, 1/16, d1=0.612, d2=0.699 g=0.991
1/1 [==============================] - 0s 30ms/step
>621, 2/16, d1=0.722, d2=0.462 g=1.056
1/1 [==============================] - 0s 30ms/step
>621, 3/16, d1=0.687, d2=0.557 g=1.073
1/1 [==============================] - 0s 42ms/step
>621, 4/16, d1=0.565, d2=0.567 g=1.032
1/1 [==============================] - 0s 32ms/step
>621, 5/16, d1=0.581, d2=0.536 g=1.070
1/1 [==============================] - 0s 32ms/step


KeyboardInterrupt: ignored

In [ ]:
# veamos los archivos generados
%ls

In [ ]:
# plt.imshow(plt.imread('generated_plot_e001.png'))
# plt.show()

In [ ]:
# plt.imshow(plt.imread('generated_plot_e020.png'))
# plt.show()

In [ ]:
# plt.imshow(plt.imread('generated_plot_e100.png'))
# plt.show()

In [ ]:
def plot_results(imgs, n_rows, n_cols, epoch, h=192, w=192):
    imgs = (imgs + 1) / 2.0
    output_img = np.zeros((n_rows*h, n_cols*w, 3), np.float)
    k = 0
    for i in range(n_rows):
        for j in range(n_cols):
            output_img[h*i:h*(i+1), w*j:w*(j+1), :] = imgs[k]
            k += 1
    plt.imshow(np.asarray(output_img*255., np.uint8))

In [ ]:
import numpy as np
latent_dim = 100
n_samples = 64

In [ ]:
# # definimos el mismo código latente para todos los tests
# x_input = generate_latent_points(latent_dim, n_samples)

# # veamos el modelo 20
# g_model = define_generator(latent_dim)
# g_model.load_weights('generator_model_020.h5')
# X = g_model.predict(x_input)
# plot_results(X, 8, 8, 100)

In [ ]:
# generamos imágenes diferentes con un código latente distinto
x_input = generate_latent_points(latent_dim, n_samples)

g_model = define_generator(latent_dim)
_last_checkpoint = sorted(os.listdir("./checkpoints/"))[-1]
g_model.load_weights(os.path.join("./checkpoints/",_last_checkpoint))
X = g_model.predict(x_input)
plot_results(X, 8, 8, 200)

# DCGAN

## Imports

In [ ]:
# # importamos las librerías necesarias
# import cv2
# import numpy as np
# from tensorflow.keras.datasets.cifar10 import load_data
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout, Activation, UpSampling2D
# # configuramos Colab para que nos muestre las imágenes más grandes
# import matplotlib.pyplot as plt
# plt.rcParams["figure.figsize"] = (10,10)

## Input images

In [ ]:
# def combine_images(generated_images):
#     num = generated_images.shape[0]
#     width = int(np.sqrt(num))
#     height = int(np.ceil(float(num)/width))
#     shape = generated_images.shape[1:3]
#     image = np.zeros((height*shape[0], width*shape[1]),
#                      dtype=generated_images.dtype)
#     for index, img in enumerate(generated_images):
#         i = int(index/width)
#         j = index % width
#         image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
#             img[:, :, 0]
#     return image

In [ ]:
# def generator_model():
#     model = Sequential([
#         Dense(1024, input_dim=100, activation='tanh'),
#         Dense(128*7*7), # This shape related to the reshape and output size
#         BatchNormalization(),
#         Activation('tanh'),
#         Reshape((7, 7, 128)),
#         UpSampling2D(size=(2, 2)),
#         Conv2D(64, (5, 5), padding='same', activation='tanh'),
#         UpSampling2D(size=(2, 2)),
#         Conv2D(3, (5, 5), padding='same', activation='tanh')
#     ])
#     return model

# generator_model().summary()

In [ ]:
# def discriminator_model():
#     model = Sequential([
#         Conv2D(64, (5, 5), input_shape=(28, 28, 3), padding='same', activation='tanh'),
#         MaxPooling2D(pool_size=(2, 2)),
#         Conv2D(128, (5, 5),activation='tanh'),
#         MaxPooling2D(pool_size=(2, 2)),
#         Flatten(),
#         Dense(1024, activation='tanh'),
#         Dense(1, activation='sigmoid')
#     ])
#     return model

# discriminator_model().summary()

In [ ]:
# def combine_model(g, d):
#     model = Sequential()
#     model.add(g)
#     model.add(d)
#     return model

In [ ]:
# def train_gan(X_train, Y_train, batch_size, epochs, g, d, save_every=500, print_every=100):

#     # Get the size of input (Z vector)
#     z_size = g.layers[0].input_shape[1]

#     # Combine discriminator on generator
#     d.trainable = False # set Discriminator to be untrainable before merging
#     d_on_g = combine_model(g, d)
#     dg_optim = RMSprop (lr=0.0005)
#     g_optim = RMSprop (lr=0.0005)
#     d_on_g.compile(loss='binary_crossentropy', optimizer=dg_optim)

#     g.compile(loss='binary_crossentropy', optimizer=g_optim)

#     # Set Discriminator to be trainable
#     d.trainable = True
#     d_optim = RMSprop (lr=0.0005)
#     d.compile(loss='binary_crossentropy', optimizer=d_optim)

#     print("Number of batches", int(X_train.shape[0]/batch_size))

#     # Start training
#     for epoch in range(epochs):
#         print("\n-------------------------------\nEpoch :", epoch)

#         for index in range(int(X_train.shape[0]/batch_size)):

#             # Randomly generate Z input
#             noise = np.random.uniform(-1, 1, size=(batch_size, z_size))

#             # Generate fake image from Z
#             generated_images = g.predict(noise, verbose=0)

#             # Take train data (real image)
#             image_batch = X_train[index*batch_size:(index+1)*batch_size]

#             if index % save_every == 0:
#                 image = combine_images(generated_images)
#                 image = image*127.5+127.5
#                 # Image.fromarray(image.astype(np.uint8)).save("train_ep"+
#                 #     str(epoch)+"_"+str(index)+".png")

#                 plt.imshow(image, cmap=plt.get_cmap('gray'))
#                 plt.axis('off')
#                 plt.show()

#             # Concatenate images to train Discriminator
#             X = np.concatenate((image_batch, generated_images))
#             y = [1] * batch_size + [0] * batch_size

#             # Train Discriminator
#             d_loss = d.train_on_batch(X, y)

#             # Randomly generate z to train Generator
#             noise = np.random.uniform(-1, 1, (batch_size, z_size))

#             # Set Discriminator to be untrainable before training the GAN (for generator)
#             d.trainable = False

#             # Train GAN (for the generator)
#             g_loss = d_on_g.train_on_batch(noise, [1] * batch_size)

#             # Print loss
#             if index % print_every == 0:
#                 print("batch %d, g_loss : %f, d_loss : %f" % (index, g_loss, d_loss))

#             # Set Discriminator to be trainable
#             d.trainable = True


#     return g, d

## Otro código

https://github.com/mitchelljy/DCGAN-Keras

In [ ]:
# %mkdir -p "/kaggle/data/models"
# %mkdir -p "/kaggle/data/output/test"
# %mkdir -p "/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/defective_resized"

In [ ]:
# !rm -r "/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/defective_resized"

In [ ]:
# from IPython.display import HTML, display
# import time

# def progress(value, max=100):
#     return HTML("""
#         <progress
#             value='{value}'
#             max='{max}',
#             style='width: 100%'
#         >
#             {value}
#         </progress>
#     """.format(value=value, max=max))

In [ ]:
# _path_folder_resized = "/kaggle/input/tyre-quality-classification/Digital images of defective and good condition tyres/defective_resized"

# _max = len(glob.glob(_path_folder+"/*.jpg"))
# out = display(progress(0, _max), display_id=True)
# for ii,img_name in enumerate(glob.glob(_path_folder+"/*.jpg")):
#   img = cv2.imread(img_name)
#   img = cv2.resize(img, (96,96), 0,0)
#   cv2.imwrite(os.path.join(_path_folder_resized,os.path.basename(img_name)), img)
#   out.update(progress(ii, _max))

In [ ]:
# from keras.models import Sequential, Model, load_model
# from keras.layers import UpSampling2D, Conv2D, Activation, BatchNormalization, Reshape, Dense, Input, LeakyReLU, Dropout, Flatten, ZeroPadding2D
# from keras.optimizers import Adam

# import glob
# import cv2
# from PIL import Image
# import numpy as np
# import os
# import argparse
# from ast import literal_eval

# # from scipy.misc import imsave


# class DCGAN:
#     def __init__(self, discriminator_path, generator_path, output_directory, img_size):
#         self.img_size = img_size
#         self.upsample_layers = 5
#         self.starting_filters = 64
#         self.kernel_size = 3
#         self.channels = 3
#         self.discriminator_path = discriminator_path
#         self.generator_path = generator_path
#         self.output_directory = output_directory

#     def build_generator(self):
#         noise_shape = (100,)

#         # This block of code can be a little daunting, but essentially it automatically calculates the required starting
#         # array size that will be correctly upscaled to our desired image size.
#         #
#         # We have 5 Upsample2D layers which each double the images width and height, so we can determine the starting
#         # x size by taking (x / 2^upsample_count) So for our target image size, 256x192, we do the following:
#         # x = (192 / 2^5), y = (256 / 2^5) [x and y are reversed within the model]
#         # We also need a 3rd dimension which is chosen relatively arbitrarily, in this case it's 64.
#         model = Sequential(name="GENERATOR")
#         model.add(
#             Dense(self.starting_filters * (self.img_size[0] // (2 ** self.upsample_layers))  *  (self.img_size[1] // (2 ** self.upsample_layers)),
#                   activation="relu", input_shape=noise_shape))
#         model.add(Reshape(((self.img_size[0] // (2 ** self.upsample_layers)),
#                            (self.img_size[1] // (2 ** self.upsample_layers)),
#                            self.starting_filters)))
#         model.add(BatchNormalization(momentum=0.8))

#         model.add(UpSampling2D())  # 6x8 -> 12x16
#         model.add(Conv2D(1024, kernel_size=self.kernel_size, padding="same"))
#         model.add(Activation("relu"))
#         model.add(BatchNormalization(momentum=0.8))

#         model.add(UpSampling2D())  # 12x16 -> 24x32
#         model.add(Conv2D(512, kernel_size=self.kernel_size, padding="same"))
#         model.add(Activation("relu"))
#         model.add(BatchNormalization(momentum=0.8))

#         model.add(UpSampling2D())  # 24x32 -> 48x64
#         model.add(Conv2D(256, kernel_size=self.kernel_size, padding="same"))
#         model.add(Activation("relu"))
#         model.add(BatchNormalization(momentum=0.8))

#         model.add(UpSampling2D())  # 48x64 -> 96x128
#         model.add(Conv2D(128, kernel_size=self.kernel_size, padding="same"))
#         model.add(Activation("relu"))
#         model.add(BatchNormalization(momentum=0.8))

#         model.add(UpSampling2D())  # 96x128 -> 192x256
#         model.add(Conv2D(64, kernel_size=self.kernel_size, padding="same"))
#         model.add(Activation("relu"))
#         model.add(BatchNormalization(momentum=0.8))

#         model.add(Conv2D(32, kernel_size=self.kernel_size, padding="same"))
#         model.add(Activation("relu"))
#         model.add(BatchNormalization(momentum=0.8))

#         model.add(Conv2D(self.channels, kernel_size=self.kernel_size, padding="same"))
#         model.add(Activation("tanh"))

#         model.summary()

#         noise = Input(shape=noise_shape)
#         img = model(noise)

#         return Model(noise, img)

#     def build_discriminator(self):

#         img_shape = (self.img_size[0], self.img_size[1], self.channels)

#         model = Sequential(name="DISCRIMINATOR")

#         model.add(Conv2D(32, kernel_size=self.kernel_size, strides=2, input_shape=img_shape, padding="same"))  # 192x256 -> 96x128
#         model.add(LeakyReLU(alpha=0.2))
#         model.add(Dropout(0.25))

#         model.add(Conv2D(64, kernel_size=self.kernel_size, strides=2, padding="same"))  # 96x128 -> 48x64
#         model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
#         model.add(LeakyReLU(alpha=0.2))
#         model.add(Dropout(0.25))
#         model.add(BatchNormalization(momentum=0.8))

#         model.add(Conv2D(128, kernel_size=self.kernel_size, strides=2, padding="same"))  # 48x64 -> 24x32
#         model.add(LeakyReLU(alpha=0.2))
#         model.add(Dropout(0.25))
#         model.add(BatchNormalization(momentum=0.8))

#         model.add(Conv2D(256, kernel_size=self.kernel_size, strides=1, padding="same"))  # 24x32 -> 12x16
#         model.add(LeakyReLU(alpha=0.2))
#         model.add(Dropout(0.25))

#         model.add(Conv2D(512, kernel_size=self.kernel_size, strides=1, padding="same"))  # 12x16 -> 6x8
#         model.add(LeakyReLU(alpha=0.2))
#         model.add(Dropout(0.25))

#         model.add(Flatten())
#         model.add(Dense(1, activation='sigmoid'))

#         model.summary()

#         img = Input(shape=img_shape)
#         validity = model(img)

#         return Model(img, validity)

#     def build_gan(self):
#         # optimizer = Adam(0.0002, 0.5)
#         optimizer = tf.keras.optimizers.legacy.Adam(0.0002, 0.5)

#         # See if the specified model paths exist, if they don't then we start training new models

#         if os.path.exists(self.discriminator_path) and os.path.exists(self.generator_path):
#             self.discriminator = load_model(self.discriminator_path)
#             self.generator = load_model(self.generator_path)
#             print("Loaded models...")
#         else:
#             self.discriminator = self.build_discriminator()
#             self.discriminator.compile(loss='binary_crossentropy',
#                                        optimizer=optimizer,
#                                        metrics=['accuracy'])

#             self.generator = self.build_generator()
#             self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

#         # These next few lines setup the training for the GAN model
#         z = Input(shape=(100,))
#         img = self.generator(z)

#         self.discriminator.trainable = False

#         valid = self.discriminator(img)

#         self.combined = Model(z, valid)
#         self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

#     def load_imgs(self, image_path):
#         X_train = []
#         for i in glob.glob(image_path):
#             # img = Image.open(i)
#             # img = np.asarray(img)
#             img = cv2.imread(i)
#             X_train.append(img)
#         return np.asarray(X_train)

#     def train(self, epochs, image_path, batch_size=32, save_interval=50):
#         self.build_gan()
#         X_train = self.load_imgs(image_path)
#         print("Training Data Shape: ", X_train.shape)

#         # Rescale images from -1 to 1
#         X_train = (X_train.astype(np.float32) - 127.5) / 127.5

#         half_batch = batch_size // 2

#         for epoch in range(epochs):


#             # Train Generator
#             noise = np.random.normal(0, 1, (batch_size, 100))
#             g_loss = self.combined.train_on_batch(noise, np.ones((batch_size, 1)))



#             # Train Discriminator
#             idx = np.random.randint(0, X_train.shape[0], half_batch)
#             imgs = X_train[idx]

#             # Sample noise and generate a half batch of new images
#             noise = np.random.normal(0, 1, (half_batch, 100))
#             gen_imgs = self.generator.predict(noise)

#             # Train the discriminator (real classified as ones and generated as zeros)
#             d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
#             d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
#             d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

#             # Print progress
#             print(f"{epoch} [D loss: {d_loss[0]} | D Accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")

#             # If at save interval => save generated image samples, save model files
#             if epoch % (save_interval) == 0:

#                 self.save_imgs(epoch)

#                 save_path = self.output_directory + "/models"
#                 if not os.path.exists(save_path):
#                     os.makedirs(save_path)
#                 self.discriminator.save(save_path + "/discrim.h5")
#                 self.generator.save(save_path + "/generat.h5")

#     def gene_imgs(self, count):
#         # Generate images from the currently loaded model
#         noise = np.random.normal(0, 1, (count, 100))
#         return self.generator.predict(noise)

#     def save_imgs(self, epoch):
#         r, c = 5, 5

#         # Generates r*c images from the model, saves them individually and as a gallery

#         imgs = self.gene_imgs(r*c)
#         imgs = 0.5 * imgs + 0.5

#         for i, img_array in enumerate(imgs):
#             path = f"{self.output_directory}/generated_{self.img_size[0]}x{self.img_size[1]}"
#             if not os.path.exists(path):
#                 os.makedirs(path)
#             # imsave(path + f"/{epoch}_{i}.png", img_array)
#             cv2.imwrite(path + f"/{epoch}_{i}.jpg", img_array)

#         nindex, height, width, intensity = imgs.shape
#         nrows = nindex // c
#         assert nindex == nrows * c
#         # want result.shape = (height*nrows, width*ncols, intensity)
#         gallery = (imgs.reshape(nrows, c, height, width, intensity)
#                   .swapaxes(1, 2)
#                   .reshape(height * nrows, width * c, intensity))

#         path = f"{self.output_directory}/gallery_generated_{self.img_size[0]}x{self.img_size[1]}"
#         if not os.path.exists(path):
#             os.makedirs(path)
#         # imsave(path + f"/{epoch}.png", gallery)
#         cv2.imwrite(path + f"/{epoch}.jpg", gallery)

#     def generate_imgs(self, count, threshold, modifier):
#         self.build_gan()

#         # Generates (count) images from the model ensuring the discriminator scores them between the threshold values
#         # and saves them

#         imgs = []
#         for i in range(count):
#             score = [0]
#             while not(threshold[0] < score[0] < threshold[1]):
#                 img = self.gene_imgs(1)
#                 score = self.discriminator.predict(img)
#             print("Image found: ", score[0])
#             imgs.append(img)

#         imgs = np.asarray(imgs).squeeze()
#         imgs = 0.5 * imgs + 0.5

#         print(imgs.shape)
#         for i, img_array in enumerate(imgs):
#             path = f"{self.output_directory}/generated_{threshold[0]}_{threshold[1]}"
#             if not os.path.exists(path):
#                 os.makedirs(path)
#             # imsave(path + f"/{modifier}_{i}.png", img_array)
#             cv2.imwrite(path + f"/{modifier}_{i}.jpg", img_array)


# if __name__ == '__main__':

#     # parser = argparse.ArgumentParser()
#     # parser.add_argument('--load_generator', help='Path to existing generator weights file', default="../data/models/generat.h5")
#     # parser.add_argument('--load_discriminator', help='Path to existing discriminator weights file', default="../data/models/discrim.h5")
#     # parser.add_argument('--data', help='Path to directory of images of correct dimensions, using *.[filetype] (e.g. *.png) to reference images', default="../data/resized/paintings_256x/*.png")
#     # parser.add_argument('--sample', help='If given, will generate that many samples from existing model instead of training', default=-1)
#     # parser.add_argument('--sample_thresholds', help='The values between which a generated image must score from the discriminator', default="(0.0, 0.1)")
#     # parser.add_argument('--batch_size', help='Number of images to train on at once', default=24)
#     # parser.add_argument('--image_size', help='Size of images as tuple (height,width). Height and width must both be divisible by (2^5)', default="(192, 256)")
#     # parser.add_argument('--epochs', help='Number of epochs to train for', default=500000)
#     # parser.add_argument('--save_interval', help='How many epochs to go between saves/outputs', default=100)
#     # parser.add_argument('--output_directory', help="Directoy to save weights and images to.", default="../data/output/test")

#     # args = parser.parse_args()

#     _load_generator = "/kaggle/data/models/generat.h5"
#     _load_discriminator = "/kaggle/data/models/discrim.h5"
#     _output_directory = "/kaggle/data/output/test"
#     _image_size = (96,96)
#     _epochs = 2000
#     _data = _path_folder_resized+"/*.jpg"
#     _batch_size = 24
#     _save_interval = 50
#     _sample = -1
#     _sample_thresholds = (0.0, 0.1)

#     dcgan = DCGAN(_load_discriminator, _load_generator, _output_directory, _image_size)
#     if _sample == -1:
#         dcgan.train(epochs=_epochs, image_path=_data, batch_size=_batch_size, save_interval=_save_interval)
#     else:
#         dcgan.generate_imgs(_sample, _sample_thresholds, "")

In [ ]:
# img = plt.imread("/kaggle/data/output/test/generated_96x96/1950_5.jpg")
# plt.imshow(img*255);
# np.max(img)